<a href="https://colab.research.google.com/github/sBx99/twitter-sentiment-analysis/blob/master/SentimentAnalysisWithTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis of Real-Time Tweets
**Try it out for yourself here!**

In [1]:
# installing all dependencies

! pip3 install tweepy
! pip3 install textblob
! pip3 install matplotlib
! pip3 install python-csv

     |████████████████████████████████| 133kB 6.7MB/s 
     |████████████████████████████████| 163kB 30.5MB/s 
  Created wheel for python-csv: filename=python_csv-0.0.11-cp36-none-any.whl size=34974 sha256=f65f8658442eda5b7575925be2899bb307e9a30ebbd37dd39b8a5f54f14edc36
  Stored in directory: /root/.cache/pip/wheels/eb/b1/ad/62414185ac29c10d529af3e285759d5d8c54e4d84d84eac337
  Created wheel for demjson: filename=demjson-2.2.4-cp36-none-any.whl size=73547 sha256=5f76469deec3bd34b7aff03fa8615a5d9c8d350bc694a5d6a653c2cf92796fbb
  Stored in directory: /root/.cache/pip/wheels/c5/d2/ab/a54fb5ea53ac3badba098160e8452fa126a51febda80440ded
  Created wheel for leven: filename=leven-1.0.4-cp36-cp36m-linux_x86_64.whl size=54673 sha256=6a6dbeeeea5d65e7ecfacd5df97f80fcbf4f9e5903e101089269b0744a3cc538
  Stored in directory: /root/.cache/pip/wheels/54/64/a5/439db671d666a50f3b3cebd2dcab3fbbab02785adf58e47552
  Created wheel for jtutils: filename=jtutils-0.0.6-cp36-none-any.whl size=7486 sha256=a340f3e91

In [0]:
import sys, tweepy, csv, re
from textblob import TextBlob
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
class SentimentAnalysis:

    def __init__(self):
        self.tweets = []
        self.tweetText = []

    def DownloadData(self):
        
        # add your twitter keys here
        consumerKey = ''
        consumerSecret = ''
        accessToken = ''
        accessTokenSecret = ''
        
        auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
        auth.set_access_token(accessToken, accessTokenSecret)
        api = tweepy.API(auth)


        searchTerm = input("Enter Keyword/Tag to search about: ")
        NoOfTerms = int(input("Enter how many tweets to search: "))


        self.tweets = tweepy.Cursor(api.search, q=searchTerm, lang = "en").items(NoOfTerms)


        csvFile = open('result.csv', 'a')
        csvWriter = csv.writer(csvFile)
        

        polarity, snegative, negative, wnegative, neutral, wpositive, positive, spositive = (0,)*8


        for tweet in self.tweets:

            self.tweetText.append(self.cleanTweet(tweet.text).encode('utf-8'))

            analysis = TextBlob(tweet.text)

            polarity += analysis.sentiment.polarity

            if (analysis.sentiment.polarity == 0):
                neutral += 1
            elif (analysis.sentiment.polarity > 0 and analysis.sentiment.polarity <= 0.3):
                wpositive += 1
            elif (analysis.sentiment.polarity > 0.3 and analysis.sentiment.polarity <= 0.6):
                positive += 1
            elif (analysis.sentiment.polarity > 0.6 and analysis.sentiment.polarity <= 1):
                spositive += 1
            elif (analysis.sentiment.polarity > -0.3 and analysis.sentiment.polarity <= 0):
                wnegative += 1
            elif (analysis.sentiment.polarity > -0.6 and analysis.sentiment.polarity <= -0.3):
                negative += 1
            elif (analysis.sentiment.polarity > -1 and analysis.sentiment.polarity <= -0.6):
                snegative += 1
            else:
                neutral = 0



        csvWriter.writerow(self.tweetText)
        csvFile.close()


        positive = self.percentage(positive, NoOfTerms)
        wpositive = self.percentage(wpositive, NoOfTerms)
        spositive = self.percentage(spositive, NoOfTerms)
        negative = self.percentage(negative, NoOfTerms)
        wnegative = self.percentage(wnegative, NoOfTerms)
        snegative = self.percentage(snegative, NoOfTerms)
        neutral = self.percentage(neutral, NoOfTerms)


        polarity = polarity / NoOfTerms
        
        # df = pd.read_csv('result.csv')
        # print("here are 10 tweets this program just collected: ")
        # df.head(10)


        print("How people are reacting on " + searchTerm + " by analyzing " + str(NoOfTerms) + " tweets.")
        print()
        print("General Report: ")

        if (polarity==0):
            print("Neutral")
        elif (polarity > 0 and polarity <= 0.3):
            print("Weak Positive")
        elif (polarity > 0.3 and polarity <= 0.6):
            print("Positive")
        elif (polarity > 0.6 and polarity <= 1):
            print("Strong Positive")
        elif (polarity > -0.3 and polarity <= 0):
            print("Weak Negative")
        elif (polarity > -0.6 and polarity <= -0.3):
            print("Negative")
        elif (polarity > -1 and polarity <= -0.6):
            print("Strong Negative")

        print('\ndetailed report: ')
        print(str(snegative), "% had a strong negative opinion about ", searchTerm)
        print(str(negative), "% had a negative opinion about ", searchTerm)
        print(str(wnegative), "% had a weak negative opinion about ", searchTerm)
        print(str(neutral), "% had a neutral opinion about ", searchTerm)
        print(str(wpositive), "% had a weak positive opinion about ", searchTerm)
        print(str(positive), "% had a positive opinion about ", searchTerm)
        print(str(spositive), "% had a strong positive opinion about ", searchTerm)

        self.plotPieChart(snegative, negative, wnegative, neutral, wpositive, positive, spositive, searchTerm, NoOfTerms)


    def cleanTweet(self, tweet):

        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w +:\ / \ / \S +)", " ", tweet).split())


    def percentage(self, part, whole):
        temp = 100 * float(part) / float(whole)
        return format(temp, '.2f')

    def plotPieChart(self, snegative, negative, wnegative, neutral, wpositive, positive, spositive, searchTerm, NoOfTerms):
            labels = ['strong negative [' + str(snegative) + '%]',
                     'negative [' + str(negative) + '%]',
                     'weak negative [' + str(wnegative) + '%]',
                     'neutral [' + str(neutral) + '%]',
                     'weak positive [' + str(wpositive) + '%]',
                     'positive [' + str(positive) + '%]',
                     'strong positive [' + str(spositive) + '%]'
                     ]
            sizes = [snegative, negative, wnegative, neutral, wpositive, positive, spositive]
            colors = ['red', 'darkorange', 'yellow', 'limegreen', 'aquamarine', 'mediumblue', 'darkviolet']
            patches, texts = plt.pie(sizes, colors=colors, startangle=90)
            plt.legend(patches, labels, loc='upper right')
            plt.title('reactions in a rainbow pie chart 🌈')
            plt.axis('equal')
            plt.tight_layout
            plt.show()

In [0]:
if __name__== "__main__":
    sentimentAnalysis = SentimentAnalysis()
    sentimentAnalysis.DownloadData()